# Practical Exam: Spectrum Shades LLC
Spectrum Shades LLC is a prominent supplier of concrete color solutions, offering a wide range of pigments and coloring systems used in various concrete applications, including decorative concrete, precast concrete, and concrete pavers. The company prides itself on delivering high-quality colorants that meet the unique needs of its diverse clientele, including contractors, architects, and construction companies.
</br></br>
The company has recently observed a growing number of customer complaints regarding inconsistent color quality in their products. The discrepancies have led to a decline in customer satisfaction and a potential increase in product returns.
By identifying and mitigating the factors causing color variations, the company can enhance product reliability, reduce customer complaints, and minimize return rates.
</br></br>
You are part of the data analysis team tasked with providing actionable insights to help Spectrum Shades LLC address the issues of inconsistent color quality and improve customer satisfaction.

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `production_data` table, and the data team have provided the following data description. 

Write a query to ensure the data matches the description provided, including identifying and cleaning all invalid values. You must match all column names and description criteria.
</br>

- You should start with the data in the file "production_data.csv".
- Your output should be a DataFrame named clean_data.
- All column names and values should match the table below.
</br>

| Column Name             | Criteria                                                                                         |
|--------------------------|--------------------------------------------------------------------------------------------------|
| batch_id | Discrete. Identifier for each batch. Missing values are not possible. |
| production_date | Date. Date when the batch was produced.|
| raw_material_supplier | Categorical. Supplier of the raw materials. (1='national_supplier', 2='international_supplier'). <br> Missing values should be replaced with 'national_supplier'.|
| pigment_type           | Nominal. Type of pigment used. ['type_a', 'type_b', 'type_c']. <br> Missing values should be replaced with 'other'. |
| pigment_quantity       | Continuous. Amount of pigment added (in kilograms) (Range: 1 - 100). <br> Missing values should be replaced with median. |
| mixing_time           | Continuous. Duration of the mixing process (in minutes). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |
| mixing_speed          | Categorical. Speed of the mixing process represented as categories: 'Low', 'Medium', 'High'.</br> Missing values should be replaced with 'Not Specified'. |
| product_quality_score | Continuous. Overall quality score of the final product (rating on a scale of 1 to 10). <br> Missing values should be replaced with mean, rounded to 2 decimal places. |


In [ ]:
# Write your answer to Task 1 here
import pandas as pd

data = pd.read_csv("production_data.csv")

data = data.dropna(subset=["batch_id"])

data["production_date"] = pd.to_datetime(data["production_date"], errors="coerce")


data["raw_material_supplier"] = data["raw_material_supplier"].fillna(1)


data["raw_material_supplier"] = data["raw_material_supplier"].replace({
    "national": 1, "National": 1, "NATIONAL": 1,
    "international": 2, "International": 2, "INTERNATIONAL": 2
})


data["pigment_type"] = data["pigment_type"].str.lower().str.strip()  
valid_pigments = ["type_a", "type_b", "type_c"]
data["pigment_type"] = data["pigment_type"].where(data["pigment_type"].isin(valid_pigments), "other")


data.loc[(data["pigment_quantity"] < 1) | (data["pigment_quantity"] > 100), "pigment_quantity"] = None
data["pigment_quantity"] = data["pigment_quantity"].fillna(data["pigment_quantity"].median())

data["mixing_time"] = data["mixing_time"].fillna(round(data["mixing_time"].mean(), 2))

data["mixing_speed"] = data["mixing_speed"].str.capitalize().str.strip()   
valid_speeds = ["Low", "Medium", "High"]
data["mixing_speed"] = data["mixing_speed"].where(data["mixing_speed"].isin(valid_speeds), "Not Specified")

data["product_quality_score"] = data["product_quality_score"].fillna(round(data["product_quality_score"].mean(), 2))

clean_data = data.copy()
print(clean_data.dtypes)
print(clean_data.head())

batch_id                          int64
production_date          datetime64[ns]
raw_material_supplier             int64
pigment_type                     object
pigment_quantity                float64
mixing_time                     float64
mixing_speed                     object
product_quality_score           float64
dtype: object
   batch_id production_date  ...  mixing_speed product_quality_score
0         1      2024-06-25  ...          High              7.165102
1         2      2023-11-23  ...          High              6.849126
2         3      2024-02-18  ...          High              5.661209
3         4      2023-11-11  ...          High              6.991735
4         5      2024-04-11  ...          High              7.095043

[5 rows x 8 columns]


# Task 2

You want to understand how the supplier type and quantity of materials affect the final product attributes.

Calculate the average `product_quality_score` and `pigment_quantity` grouped by `raw_material_supplier`.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named aggregated_data.
- It should include the three columns: `raw_material_supplier`, `avg_product_quality_score`, and `avg_pigment_quantity`.
- Your answers should be rounded to 2 decimal places.


In [ ]:
# Write your answer to Task 2 here
import pandas as pd

data = pd.read_csv("production_data.csv")

aggregated_data = (
    data.groupby("raw_material_supplier")[["product_quality_score", "pigment_quantity"]]
    .mean()
    .reset_index()
)

aggregated_data = aggregated_data.rename(
    columns={
        "product_quality_score": "avg_product_quality_score",
        "pigment_quantity": "avg_pigment_quantity"
    }
)

aggregated_data["avg_product_quality_score"] = aggregated_data["avg_product_quality_score"].round(2)
aggregated_data["avg_pigment_quantity"] = aggregated_data["avg_pigment_quantity"].round(2)


print(aggregated_data)


   raw_material_supplier  avg_product_quality_score  avg_pigment_quantity
0                      1                       8.02                 44.73
1                      2                       5.97                 34.91


# Task 3

To get more insight into the factors behind product quality, you want to filter the data to see an average product quality score for a specified set of results.

Identify the average `product_quality_score` for batches with a `raw_material_supplier` of 2 and a `pigment_quantity` greater than 35 kg.

Write a query to return the average `avg_product_quality_score` for these filtered batches. Use the original production data table, not the output of Task 2.

- You should start with the data in the file 'production_data.csv'. 
- Your output should be a DataFrame named pigment_data.
- It should consist of a 1-row DataFrame with 3 columns: `raw_material_supplier`, `pigment_quantity`, and `avg_product_quality_score`.
- Your answers should be rounded to 2 decimal places where appropriate.


In [ ]:
# Write your answer to Task 3 here

import pandas as pd

data = pd.read_csv("production_data.csv")

filtered = data[
    (data["raw_material_supplier"] == 2) & (data["pigment_quantity"] > 35)
]

avg_quality = round(filtered["product_quality_score"].mean(), 2)

pigment_data = pd.DataFrame({
    "raw_material_supplier": [2],
    "pigment_quantity": [">35"],
    "avg_product_quality_score": [avg_quality]
})

print(pigment_data)


   raw_material_supplier pigment_quantity  avg_product_quality_score
0                      2              >35                       5.97


# Task 4

In order to proceed with further analysis later, you need to analyze how various factors relate to product quality. Start by calculating the mean and standard deviation for the following columns: `pigment_quantity`, and `product_quality_score`. </br> These statistics will help in understanding the central tendency and variability of the data related to product quality.
</br> </br >
Next, calculate the Pearson correlation coefficient between the following variables: `pigment_quantity`, and `product_quality_score`.
</br>
These correlation coefficients will provide insights into the strength and direction of the relationships between the factors and overall product quality.


- You should start with the data in the file 'production_data.csv'.
- Calculate the mean and standard deviation for the columns pigment_quantity and product_quality_score as: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`.
- Calculate the Pearson correlation coefficient between pigment_quantity and product_quality_score as: `corr_coef`
- Your output should be a DataFrame named product_quality.
- It should include the columns: `product_quality_score_mean`, `product_quality_score_sd`, `pigment_quantity_mean`, `pigment_quantity_sd`, `corr_coef`.
- Ensure that your answers are rounded to 2 decimal places.


In [ ]:
# Write your answer to Task 4 here
import pandas as pd

data = pd.read_csv("production_data.csv")

median_pigment = data['pigment_quantity'].median()

data['pigment_quantity'] = data['pigment_quantity'].apply(

lambda x: x if 1 <= x <= 100 else np.nan

)

data['pigment_quantity'].fillna(median_pigment, inplace=True)

mean_quality = round(data['product_quality_score'].mean(), 2)

data['product_quality_score'].fillna(mean_quality, inplace=True)

pigment_quantity_mean = round(data['pigment_quantity'].mean(), 2)

pigment_quantity_sd = round(data['pigment_quantity'].std(), 2)

product_quality_score_mean = round(data['product_quality_score'].mean(), 2)

product_quality_score_sd = round(data['product_quality_score'].std(), 2)

corr_coef = round(data['pigment_quantity'].corr(data['product_quality_score']), 2)

product_quality = pd.DataFrame([{

'product_quality_score_mean': product_quality_score_mean,

'product_quality_score_sd': product_quality_score_sd,

'pigment_quantity_mean': pigment_quantity_mean,

'pigment_quantity_sd': pigment_quantity_sd,

'corr_coef': corr_coef

}])

print(product_quality)

   product_quality_score_mean  ...  corr_coef
0                        6.68  ...       0.49

[1 rows x 5 columns]
